In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install datasets
!pip install gdown

In [2]:
import gdown

url = 'https://drive.google.com/uc?export=download&id=1zZC7jGTY6kPUIVdAMhmXvad_-K1dD88Z'
output = 'pytorch_model.bin'
gdown.download(url, output, quiet=False)
url = 'https://drive.google.com/uc?export=download&id=1jbbUtwgwoSQgGnXxzTh-nMReVzEU7ZTU&confirm=t&uuid=d79e2e78-51de-466f-9ceb-3944606141a2&at=AKKF8vwcgi95TGSnSQUNCKx4NTqS:1682865249145'
output = 'output.jsonl'
gdown.download(url, output, quiet=False)
url = 'https://drive.google.com/uc?export=download&id=1E0hqkgPgl9NA18OmFZrrvHJRdl26dbzj'
output = 'config.json'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=1zZC7jGTY6kPUIVdAMhmXvad_-K1dD88Z
To: /content/pytorch_model.bin
100%|██████████| 1.13G/1.13G [00:11<00:00, 94.8MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1jbbUtwgwoSQgGnXxzTh-nMReVzEU7ZTU&confirm=t&uuid=d79e2e78-51de-466f-9ceb-3944606141a2&at=AKKF8vwcgi95TGSnSQUNCKx4NTqS:1682865249145
To: /content/output.jsonl
100%|██████████| 167M/167M [00:02<00:00, 79.6MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1E0hqkgPgl9NA18OmFZrrvHJRdl26dbzj
To: /content/config.json
100%|██████████| 610/610 [00:00<00:00, 69.0kB/s]


'config.json'

In [4]:
import os
os.mkdir('model')

In [5]:
import shutil
source = '/content/config.json' 
destination = '/content/model/config.json'
shutil.move(source, destination)
source = '/content/pytorch_model.bin'
destination = '/content/model/pytorch_model.bin'
shutil.move(source, destination)

'/content/model/config.json'

In [7]:
from transformers import XGLMTokenizer, XGLMForCausalLM, GenerationConfig
import torch
def load_model(base_model:str="facebook/xglm-564M"):
    tokenizer = XGLMTokenizer.from_pretrained(base_model)
    new_tokens = ['<human>:', '<bot>:']
    tokenizer.add_tokens(list(new_tokens))
    model = XGLMForCausalLM.from_pretrained('/content/model/', local_files_only=True,torch_dtype=torch.float16)
    return model,tokenizer
model,tokenizer = load_model()

In [54]:
from datasets import load_dataset,Dataset
datasets = load_dataset('json',data_files = 'output.jsonl')
datasets = Dataset.from_dict(datasets['train'][:10_000])
datasets

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['Background:', '<human>:', '<bot>:'],
    num_rows: 10000
})

In [55]:
def format_prompt(prompt):
    return {'prompt':f"{prompt['Background:']}"}
# datasets = datasets['train']
datasets = datasets.map(format_prompt,remove_columns=['Background:'])
datasets = datasets.map(lambda x:{'token':len(tokenizer.tokenize(x['prompt']))})
datasets = datasets.filter(lambda x:x['token']<255)
# datasets = datasets.map(preprocess,remove_columns=['prompt','token']) 
datasets

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['<human>:', '<bot>:', 'prompt', 'token'],
    num_rows: 6271
})

In [57]:
prompt = datasets['prompt'][1]
print(prompt)

 The 2012 presidential campaign of Jill Stein was announced on October 24 2011 Jill Stein a physician from Massachusetts gave indication in August 2011 that she was considering running for President of the United States with the Green Party in the 2012 national election She wrote in a published questionnaire that she had been asked to run by a number of Green activists and felt compelled to consider the possibility after the USdebtceiling crisis which she called the Presidents astounding attack on Social Security Medicare and Medicaid  a betrayal of the public interest Stein received the presidential nomination of Green Party at its nominating convention in Baltimore on July 14 2012 The campaign received enough contributions to qualify for primary season federal matching funds from the Federal Election Commission and on July 11 2012 Stein selected antipoverty activist Cheri Honkala as her running mate for the Green vicepresidential nomination <human>: 


In [ ]:
import torch
generation_config = GenerationConfig(
        temperature=0.1,
        top_p=0.75,
        top_k=40,
        num_beams=4,
    )
device = torch.device('cuda')
model.to(device)

In [103]:
import re
def gen(prompt,generation_config=generation_config):
  prompt = f'{prompt} <human>:'
  inputs = tokenizer(prompt, return_tensors="pt")
  input_ids = inputs["input_ids"]
  input_ids = input_ids.to(device)
  with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )
  output_1_ids = generation_output.sequences[0]
  output = tokenizer.decode(output_1_ids)  
  bot_prompt = re.split(tokenizer.eos_token,output)[1]
  bot_prompt = f'{bot_prompt} <bot>:'

  inputs = tokenizer(bot_prompt, return_tensors="pt")
  input_ids = inputs["input_ids"]
  input_ids = input_ids.to(device)
  with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )
  s = generation_output.sequences[0]
  output = tokenizer.decode(s)
  res = re.split('<human>: | <bot>:',output)
  return f'<human>: {res[1]} \n<bot>: {res[2]}'

In [104]:
sample = """
Doraemon  is a Japanese manga series written and illustrated 
by Fujiko F. Fujio. The manga was first serialized in December 1969, 
with its 1,345 individual chapters compiled into 45 tankōbon volumes 
and published by Shogakukan from 1970 to 1996. The story revolves around 
an earless robotic cat named Doraemon, who travels back in time from the 
22nd century to aid a boy named Nobita Nobi."""

In [105]:
response = gen(sample)
print(response)

<human>: Tell me about Doraemon Doraemon is a Japanese manga series written and illustrated by Fujiko F. Fujio. The manga was first serialized in December 1969 
<bot>:  Doraemon is a Japanese manga series written and illustrated by Fujiko F. Fujio</s>
